In [ ]:
import torch.nn as nn
import torch
import numpy as np
import torch.optim as optim
import pandas as pd
from scipy.signal import savgol_filter
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [ ]:
def getStates():
    ds = pd.read_csv('data/test.csv')
    states = ds['Province_State'][:50].values
    return states

In [ ]:
degree = 3
states = getStates()
start = 110
alpha = .1
window = 17

In [ ]:
feature = 'Deaths'
res = []
for i in range(len(states)):
    state = states[i]
    ds = pd.read_csv('data/train.csv')
    ds = ds[ds['Province_State'] == state]
    value = ds[feature].values
    value = savgol_filter(value.reshape(-1), window, degree)
    combine = [[value[i]] for i in range(len(value))]
    
    y = np.array([value[start:]]).reshape(-1, 1)
    x = [[i + start] for i in range(142-start)]
    
#     model = pl.make_pipeline(
#         sp.PolynomialFeatures(1), 
#         lm.LinearRegression()  
#     )
#     model = lm.Lasso(alpha=.05, fit_intercept=True)
    model = Pipeline([
        ("poly", PolynomialFeatures(degree=2)),
        ("lasso_reg", Ridge(alpha=alpha))
    ])
    model.fit(x, y)
    
    x_test = [[i + 142] for i in range(26)]
    
    y_hat = model.predict(x_test)
    res.append(y_hat)

In [ ]:
rerange = []
for i in range(len(res[0])):
    for j in range(len(res)):
        rerange.append(res[j][i])
df = pd.DataFrame(rerange)
df.to_csv("{}.csv".format(feature),index=False,sep=',')

In [ ]:
confirmed_test = pd.read_csv('Confirmed.csv')
real = pd.read_csv('data/test.csv')
deaths_test = pd.read_csv('deaths.csv')

In [ ]:
confirmed = confirmed_test['0'].values
deaths = deaths_test['0'].values

In [ ]:
real_confirmed = real['Confirmed'].values
real_deaths = real['Deaths'].values

In [ ]:
def score():
    total = 0
    for i in range(len(confirmed)):
        n_c = abs(confirmed[i] - real_confirmed[i]) / real_confirmed[i]
        n_d = abs(deaths[i] - real_deaths[i]) / real_deaths[i]
        total += n_d

    total = total * 100 / (1 * len(confirmed))
    print(total)
score()

3.4389569542523803
